In [1]:
import investpy
import re
import pandas as pd
import numpy as np

### get sector, ticker, counter_id from malaysiastockbiz (msb)

In [2]:
def load_data():
    KL = "https://www.malaysiastock.biz/Listed-Companies.aspx?type=S&s1="

    df = pd.concat(
        [pd.read_html(f"{KL}{i+1}")[8] for i in range(30)]
    )

    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)

    df["counter_id"] = df["Company"].map(lambda x: re.search("\((.*?)\)", x).group(1))
    df["company_slug"] = df["Company"].map(lambda x: re.search("^([^\s]+)", x).group(1))
    df["company_name"] = df["Company"].map(lambda x: re.search("\)(.+)$", x).group(1))

    df["is_ace"] = df["company_name"].map(lambda x: True if x[:3] == "ACE" else False)
    df["is_etf"] = df["company_name"].map(lambda x: True if x[:5] == "ETFMY" else False)
    df["is_leap"] = df["company_name"].map(lambda x: True if x[:4] == "LEAP" else False)
    df["is_main"] = df["company_name"].map(lambda x: True if x[:4] == "MAIN" else False)

    df["company_name"] = df["company_name"].map(lambda x: x[3:] if x[:3] == "ACE" else x)
    df["company_name"] = df["company_name"].map(lambda x: x[5:] if x[:5] == "ETFMY" else x)
    df["company_name"] = df["company_name"].map(lambda x: x[4:] if x[:4] == "LEAP" else x)
    df["company_name"] = df["company_name"].map(lambda x: x[4:] if x[:4] == "MAIN" else x)

# manual fix to align all as int
    df["counter_id"] = df["counter_id"].replace({"5235SS": "5235"})
    
# drop null columns    
    df.drop(columns = ["Company", "Shariah"], inplace=True, axis=0)

# better to get market cap,PE, DY from investpy    
#     df = df[["company_name", "company_slug", "counter_id", "Market Cap", "Last Price", "PE", 
#              "DY", "ROE", "is_main","is_ace", "is_leap", "is_equity",  "Sector"]]
    
    df = df[["company_name", "company_slug", "counter_id", "Sector", 
             "is_main","is_ace", "is_leap", "is_etf"]]
    
    df.drop_duplicates(inplace=True)
    return df

In [3]:
msb = load_data()

### Read counter_id crawled

In [4]:
stocks = pd.read_json('counter_id.json')
stocks.columns = stocks.columns.str.lower()

In [5]:
#erasa "PA" - Preference share from counter_id and fill in leading zeros 
stocks['counter_id'] = stocks['counter_id'].map(lambda x: re.search("\d{1,}", x).group(0))
stocks['counter_id'] = stocks['counter_id'].apply(lambda x: x.zfill(4))

In [6]:
stocks.set_index("counter_id", inplace=True)
msb.set_index("counter_id", inplace=True)

### Merge

In [7]:
merge = stocks.join(msb, on="counter_id")
merge.reset_index(inplace=True)
merge

,counter_id,ticker,company_name,company_slug,Sector,is_main,is_ace,is_leap,is_etf
0,5090,MDCH,MEDIA CHINESE INTERNATIONAL LIMITED,MEDIAC,Media,True,False,False,False
1,1015,AMMB,AMMB HOLDINGS BERHAD,AMBANK,Banking,True,False,False,False
2,1023,CIMB,CIMB GROUP HOLDINGS BERHAD,CIMB,Banking,True,False,False,False
3,1066,RHBC,RHB BANK BERHAD,RHBBANK,Banking,True,False,False,False
4,1082,HLCB,HONG LEONG FINANCIAL GROUP BERHAD,HLFG,Banking,True,False,False,False
...,...,...,...,...,...,...,...,...,...
924,0208,GREA,GREATECH TECHNOLOGY BERHAD,GREATEC,Technology Equipment,False,True,False,False
925,0207,MEST,MESTRON HOLDINGS BERHAD,MESTRON,Metals,False,True,False,False
926,0022,PARL,PARLO BERHAD,PARLO,"Travel, Leisure & Hospitality",False,True,False,False
927,0213,MTAG,MTAG GROUP BERHAD,MTAG,Industrial Services,False,True,False,False


In [8]:
print(f"Main: {len(merge[merge.is_main==True])}")
print(f"Ace: {len(merge[merge.is_ace==True])}")
print(f"Leap: {len(merge[merge.is_leap==True])}")
print(f"ETF: {len(merge[merge.is_etf==True])}")
print(f"Total: {(len(merge[merge.company_slug.notnull()]))}")

# manually checked the list below in market, no longer listed
print(f"Delisted: {(len(merge[merge.company_slug.isnull()]))}")

Main: 791
Ace: 121
Leap: 6
ETF: 0
Total: 918
Delisted: 11


### number of stocks from bursa

https://www.bursamalaysia.com/listing/listing_resources/ipo/listing_statistic

In [ ]:
Year	Main Ace LEAP	Total
2020	771	130	32	933
2019	772	129	28	929
2018	783	119	13	915
2017	788	115	2	905

### other useful stuffs

In [9]:
investpy.get_stock_company_profile('cimb', country='malaysia')

{'url': 'https://www.investing.com/equities/bumiputra---commerce-holdings-bhd-company-profile',
 'desc': "CIMB Group Holdings Berhad is a Malaysia-based provider of banking products and financial services. The Company's major business activities are categorized into three segments: Consumer Banking, Wholesale Banking and Commercial Banking. Its Consumer Banking segment provides regular banking solutions to individual customers, and it covers both conventional and Islamic financial products and services, such as residential and non-residential property loans, motor vehicle financing, credit cards and wealth management. Its Commercial Banking segment offers various products and services to small and medium-scale enterprises (SMEs) that include core banking credit facilities, trade financing, remittance and foreign exchange, together with general deposit products. Its Wholesale Banking segment comprises Investment Banking, Corporate Banking, Treasury and Markets, Transaction Banking, as w

In [10]:
investpy.stocks.get_stock_information("cimb", country='malaysia')

,Stock Symbol,Prev. Close,Todays Range,Revenue,Open,52 wk Range,EPS,Volume,Market Cap,Dividend (Yield),Average Vol. (3m),P/E Ratio,Beta,1-Year Change,Shares Outstanding,Next Earnings Date
0,CIMB,3.41,3.39 - 3.47,1.611000e+10,3.41,3.09 - 5.45,0.47,19432900.0,3.404000e+10,0.26 (7.62%),21340510.0,7.25,1.92,- 34.92%,9.922961e+09,20/05/2020


In [11]:
investpy.technical.technical_indicators(name='CIMB', country='malaysia', product_type='stock', interval='daily')

,technical_indicator,value,signal
0,RSI(14),43.2300,sell
1,"STOCH(9,6)",21.1220,sell
2,STOCHRSI(14),39.6720,sell
3,"MACD(12,26)",-0.1100,sell
4,ADX(14),23.6200,buy
5,Williams %R,-78.7230,sell
6,CCI(14),-75.8639,sell
7,ATR(14),0.0979,high_volatility
8,Highs/Lows(14),-0.0271,sell
9,Ultimate Oscillator,44.0530,sell


In [12]:
investpy.news.get_calendar(countries=['malaysia'])

,id,date,time,zone,currency,importance,event,actual,forecast,previous
0,405642,05/05/2020,15:00,malaysia,MYR,medium,Interest Rate,2.00%,2.00%,2.50%
